# Comparative Subject AE

In [ ]:
# !pip install torchinfo
import torch
import torch.nn as nn
from torchinfo import summary
import matplotlib.pyplot as plt
import os
import gc
# os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np
import time
import CompMethods_V3 as TP
import DataSetting_v5 as DS

In [ ]:
date = '20240917'
name = f'{date}_COMP_sub'
data_path = ['../dataset/Door_EXP/A208']
level = 'subject'

data_organizer = DS.DataOrganizer(name, data_path, level)
data_organizer.load()

### AE

In [ ]:
preprocess = DS.Preprocess(new_size=(128, 128))
data_organizer.load_plan('../dataset/Door_EXP/subject_r0.2.pkl')

def run():
    for i in range(6):
        data_organizer.gen_plan()
        train_loader, valid_loader, test_loader, current_test = data_organizer.gen_loaders(mode='s', num_workers=2)
        AE_trainer = TP.CompTrainer(name='AE', mask=False,
                                        networks=[TP.AutoEncoder(latent_dim=128, middle_dim=512*7*75)],
                                        lr=1e-4, epochs=10, cuda=gpu,
                                        preprocess = preprocess,
                                        loss_optimizer = {'LOSS': [torch.optim.Adam, 1e-4]},
                                        notion=f"{date}_COMP_sub_{current_test}_AE",
                                        train_loader=train_loader, valid_loader=valid_loader, test_loader=test_loader)
        ae_trained = AE_trainer.schedule(lr_decay=True)
        break
        # del AE_trainer
        # del ae_trained
        # torch.cuda.empty_cache()
        # gc.collect()
    return AE_trainer

In [ ]:
gpu = 6
torch.cuda.set_device(gpu)
trainer = run()